# **Machine Learning on Big Data (CN7030) CRWK 23-24 Term B [60% weighting]**
# **Group ID: [Group13]**
1.   Student 1: Philip Acquaye-Mensah 2640756
2.   Student 2: Mohamed Jareer MOHAMED  ZEENAM 2596353

---

If you want to add comments on your group work, please write it here for us:


# **Initiate and Configure Spark**

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, MinMaxScaler, RobustScaler, Imputer
from pyspark.ml.classification import LogisticRegression, NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.regression import LinearRegression


# Initialize SparkSession
spark = SparkSession.builder \
                    .appName("Group 13 CRWK") \
                    .master("local[*]") \
                    .config("spark.executor.memory", "4g") \
                    .config("spark.driver.memory", "2g") \
                    .config("spark.executor.cores", "2") \
                    .config("spark.sql.inMemoryColumnarStorage.compressed", "true") \
                    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/16 10:48:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


---
# **Task 1 - Data Loading and Preprocessing (15 marks)**
---

In [2]:
#Identify the student who made a contribution and mention their name in the appropriate section of the code.

## Philip Acquaye-Mensah

# Load the compressed file as a text file
df = spark.read.csv("CourseWork_Dataset_Machine_Learning.csv", header = True)
# Display the DataFrame
df.show(20, truncate= False)

# more info
print(df.count(), df.rdd.getNumPartitions())

24/04/16 10:49:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+--------+-------------------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+---------------+-------------+----------------+----------------+------------+------------+-----------+----------------+----------------+-----------+-----------+-----------+----------------+----------------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+------------+-----------+-----------+--------------+--------------+----------------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+--------------+----------------+----------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+----------------+----------------+----------------+--------------

1048575 4


In [3]:
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType

# Exclude 'Label' column from casting
columns_to_cast = [column for column in df.columns if column != 'Label']

for column in columns_to_cast:
    new_column = column.replace(' ', '_').replace('/', '_per_')
    df = df.withColumnRenamed(column, new_column).withColumn(new_column, col(new_column).cast(FloatType()))

In [4]:
# Check Multi class label
# Label is the column named Label
df.select("Label").distinct().show(20)

+--------------+
|         Label|
+--------------+
|SSH-Bruteforce|
|        Benign|
|FTP-BruteForce|
+--------------+



In [5]:
from pyspark.sql.functions import when, col

# Convert "Label" into a binary classification where "Benign" is 0 and any attack is 1
df = df.withColumn("Label_binary", when(col("Label") == "Benign", 0).otherwise(1))

# Display the updated DataFrame to verify the transformation
df.select("Label", "Label_binary").distinct().show()

+--------------+------------+
|         Label|Label_binary|
+--------------+------------+
|FTP-BruteForce|           1|
|SSH-Bruteforce|           1|
|        Benign|           0|
+--------------+------------+



In [6]:
# Drop Label column because of new binary label and Timestamp column
df = df.drop("Label","Timestamp")

# Print the schema to see the data types and structure
df.printSchema()

root
 |-- Dst_Port: float (nullable = true)
 |-- Protocol: float (nullable = true)
 |-- Flow_Duration: float (nullable = true)
 |-- Tot_Fwd_Pkts: float (nullable = true)
 |-- Tot_Bwd_Pkts: float (nullable = true)
 |-- TotLen_Fwd_Pkts: float (nullable = true)
 |-- TotLen_Bwd_Pkts: float (nullable = true)
 |-- Fwd_Pkt_Len_Max: float (nullable = true)
 |-- Fwd_Pkt_Len_Min: float (nullable = true)
 |-- Fwd_Pkt_Len_Mean: float (nullable = true)
 |-- Fwd_Pkt_Len_Std: float (nullable = true)
 |-- Bwd_Pkt_Len_Max: float (nullable = true)
 |-- Bwd_Pkt_Len_Min: float (nullable = true)
 |-- Bwd_Pkt_Len_Mean: float (nullable = true)
 |-- Bwd_Pkt_Len_Std: float (nullable = true)
 |-- Flow_Byts_per_s: float (nullable = true)
 |-- Flow_Pkts_per_s: float (nullable = true)
 |-- Flow_IAT_Mean: float (nullable = true)
 |-- Flow_IAT_Std: float (nullable = true)
 |-- Flow_IAT_Max: float (nullable = true)
 |-- Flow_IAT_Min: float (nullable = true)
 |-- Fwd_IAT_Tot: float (nullable = true)
 |-- Fwd_IAT_Mean:

In [7]:
# Check for missing values in each column
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------+--------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+-------------+------------+------------+------------+-----------+------------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+--------------+--------------+-----------+-----------+------------+-----------+-----------+------------+------------+------------+------------+------------+------------+--------------+------------+-----------------+------------+----------------+----------------+------------------+------------------+----------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------

In [8]:
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType


# Now, use the Imputer on the renamed column
imputer = Imputer(
    inputCols=["Flow_Byts_per_s"],
    outputCols=["Flow_Byts_s_imputed"]
).setStrategy("median")

df = imputer.fit(df).transform(df)

# Drop the 'Flow_Byts_per_s' column
df = df.drop('Flow_Byts_per_s')

# Print Schema again to check the changes
df.printSchema()

root
 |-- Dst_Port: float (nullable = true)
 |-- Protocol: float (nullable = true)
 |-- Flow_Duration: float (nullable = true)
 |-- Tot_Fwd_Pkts: float (nullable = true)
 |-- Tot_Bwd_Pkts: float (nullable = true)
 |-- TotLen_Fwd_Pkts: float (nullable = true)
 |-- TotLen_Bwd_Pkts: float (nullable = true)
 |-- Fwd_Pkt_Len_Max: float (nullable = true)
 |-- Fwd_Pkt_Len_Min: float (nullable = true)
 |-- Fwd_Pkt_Len_Mean: float (nullable = true)
 |-- Fwd_Pkt_Len_Std: float (nullable = true)
 |-- Bwd_Pkt_Len_Max: float (nullable = true)
 |-- Bwd_Pkt_Len_Min: float (nullable = true)
 |-- Bwd_Pkt_Len_Mean: float (nullable = true)
 |-- Bwd_Pkt_Len_Std: float (nullable = true)
 |-- Flow_Pkts_per_s: float (nullable = true)
 |-- Flow_IAT_Mean: float (nullable = true)
 |-- Flow_IAT_Std: float (nullable = true)
 |-- Flow_IAT_Max: float (nullable = true)
 |-- Flow_IAT_Min: float (nullable = true)
 |-- Fwd_IAT_Tot: float (nullable = true)
 |-- Fwd_IAT_Mean: float (nullable = true)
 |-- Fwd_IAT_Std: flo

In [9]:
df.describe("Flow_Pkts_per_s").show(10)

+-------+---------------+
|summary|Flow_Pkts_per_s|
+-------+---------------+
|  count|        1048575|
|   mean|       Infinity|
| stddev|            NaN|
|    min|  -0.0010297608|
|    max|       Infinity|
+-------+---------------+



In [41]:
from pyspark.sql.functions import col, isnan

# Iterate through each column in the DataFrame
for column in df.columns:
    # Filter and count rows with null values
    null_count = df.filter(col(column).isNull()).count()
    if null_count > 0:
        print(f"Column '{column}' has {null_count} rows with null values.")
    
    # Filter and count rows with NaN values
    nan_count = df.filter(isnan(col(column))).count()
    if nan_count > 0:
        print(f"Column '{column}' has {nan_count} rows with NaN values.")
    
    # Filter and count rows with negative values
    negative_count = df.filter(col(column) < 0).count()
    if negative_count > 0:
        print(f"Column '{column}' has {negative_count} rows with negative values.")

    # Filter and count rows with infinite values
    infinite_count = df.filter(col(column) == float('inf')).count()
    if infinite_count > 0:
        print(f"Column '{column}' has {infinite_count} rows with infinite values.")

    #


In [26]:
# from pyspark.sql.functions import col, isnan

# # List of columns with less than 6 problematic rows to be dropped
# columns_to_drop_rows = [
#     "Flow_Duration", "Flow_Pkts_per_s", "Flow_IAT_Mean", "Flow_IAT_Max", "Flow_IAT_Min",
#     "Fwd_IAT_Tot", "Fwd_IAT_Mean", "Fwd_IAT_Max", "Fwd_IAT_Min"
# ]

# # Create a filter condition that selects rows without problematic values in any of the specified columns
# condition = None
# for column in columns_to_drop_rows:
#     current_condition = (col(column).isNotNull()) & (~isnan(col(column))) & (col(column) >= 0)
#     condition = current_condition if condition is None else (condition & current_condition)

# # Apply the filter to drop rows with any problematic values in the specified columns
# df = df.filter(condition)

# print(f"Rows remaining after dropping problematic values: {df.count()}")
# df.printSchema()


Rows remaining after dropping problematic values: 1044751
root
 |-- Dst_Port: float (nullable = true)
 |-- Protocol: float (nullable = true)
 |-- Flow_Duration: float (nullable = true)
 |-- Tot_Fwd_Pkts: float (nullable = true)
 |-- Tot_Bwd_Pkts: float (nullable = true)
 |-- TotLen_Fwd_Pkts: float (nullable = true)
 |-- TotLen_Bwd_Pkts: float (nullable = true)
 |-- Fwd_Pkt_Len_Max: float (nullable = true)
 |-- Fwd_Pkt_Len_Min: float (nullable = true)
 |-- Fwd_Pkt_Len_Mean: float (nullable = true)
 |-- Fwd_Pkt_Len_Std: float (nullable = true)
 |-- Bwd_Pkt_Len_Max: float (nullable = true)
 |-- Bwd_Pkt_Len_Min: float (nullable = true)
 |-- Bwd_Pkt_Len_Mean: float (nullable = true)
 |-- Bwd_Pkt_Len_Std: float (nullable = true)
 |-- Flow_IAT_Mean: float (nullable = true)
 |-- Flow_IAT_Std: float (nullable = true)
 |-- Flow_IAT_Max: float (nullable = true)
 |-- Flow_IAT_Min: float (nullable = true)
 |-- Fwd_IAT_Tot: float (nullable = true)
 |-- Fwd_IAT_Mean: float (nullable = true)
 |-- Fwd_

In [27]:
# from pyspark.sql.functions import mean

# # Additional columns with problematic rows that need mean imputation
# additional_columns_to_impute = ["Init_Fwd_Win_Byts", "Init_Bwd_Win_Byts"]

# for column in additional_columns_to_impute:
#     # Calculate mean excluding nulls, NaNs, and negative values
#     mean_value = df.filter((col(column).isNotNull()) & (~isnan(col(column))) & (col(column) >= 0)) \
#                    .agg(mean(col(column)).alias('mean')) \
#                    .collect()[0]['mean']

#     # Impute problematic values with the calculated mean
#     df = df.withColumn(column, when((col(column).isNull()) | (isnan(col(column))) | (col(column) < 0), mean_value) \
#                                 .otherwise(col(column)))

#     print(f"Imputed '{column}' with mean value: {mean_value:.2f}")


Imputed 'Init_Fwd_Win_Byts' with mean value: 10534.79


Imputed 'Init_Bwd_Win_Byts' with mean value: 6165.01


In [ ]:
from pyspark.sql.functions import when, col

# List of columns to replace Infinity values with NaN
columns = ["Flow_Byts_s_imputed", "Flow_Pkts_per_s"]

#! Loop through each column and replace Infinity with NaN
for col_name in columns:
    df = df.withColumn(col_name, when(col(col_name) == float('inf'), float('nan')).otherwise(col(col_name)))
    
#! Replace negative values with zero for all columns in the DataFrame
for col_name in df.columns:
    df = df.withColumn(col_name, when(df[col_name] < 0, 0).otherwise(df[col_name]))


output_cols = ["Flow_Byts_s_nan_imputed", "Flow_Pkts_per_s_nan_imputed"]


#! impute nan values with mean
imputer = Imputer(
    inputCols=columns,
    outputCols=output_cols
).setStrategy("median")



df = imputer.fit(df).transform(df)

df.describe("Flow_Byts_s_nan_imputed").show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Flow_Byts_s_imputed` cannot be resolved. Did you mean one of the following? [`Flow_Byts_s_nan_imputed`, `Flow_Pkts_per_s_nan_imputed`, `Flow_IAT_Min`, `Flow_IAT_Std`, `Bwd_Byts_per_b_Avg`].;
'Project [Dst_Port#22843, Protocol#22923, Flow_Duration#23003, Tot_Fwd_Pkts#23083, Tot_Bwd_Pkts#23163, TotLen_Fwd_Pkts#23243, TotLen_Bwd_Pkts#23323, Fwd_Pkt_Len_Max#23403, Fwd_Pkt_Len_Min#23483, Fwd_Pkt_Len_Mean#23563, Fwd_Pkt_Len_Std#23643, Bwd_Pkt_Len_Max#23723, Bwd_Pkt_Len_Min#23803, Bwd_Pkt_Len_Mean#23883, Bwd_Pkt_Len_Std#23963, Flow_IAT_Mean#24123, Flow_IAT_Std#24203, Flow_IAT_Max#24283, Flow_IAT_Min#24363, Fwd_IAT_Tot#24443, Fwd_IAT_Mean#24523, Fwd_IAT_Std#24603, Fwd_IAT_Max#24683, Fwd_IAT_Min#24763, ... 56 more fields]
+- Project [Dst_Port#22843, Protocol#22923, Flow_Duration#23003, Tot_Fwd_Pkts#23083, Tot_Bwd_Pkts#23163, TotLen_Fwd_Pkts#23243, TotLen_Bwd_Pkts#23323, Fwd_Pkt_Len_Max#23403, Fwd_Pk

In [29]:
df = df.drop('Flow_Byts_s_imputed', 'Flow_Pkts_per_s')

df.show(10, truncate=False)

+--------+--------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+-------------+------------+------------+------------+------------+------------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+--------------+--------------+-----------+-----------+------------+-----------+-----------+------------+------------+------------+------------+------------+------------+--------------+------------+-----------------+------------+----------------+----------------+------------------+------------------+----------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+-----------------+----------------+------

In [30]:
# Check for missing values again
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------+--------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+-------------+------------+------------+------------+-----------+------------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+--------------+--------------+-----------+-----------+------------+-----------+-----------+------------+------------+------------+------------+------------+------------+--------------+------------+-----------------+------------+----------------+----------------+------------------+------------------+----------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+-----------------+----------------+-------

In [14]:
# from pyspark.sql.functions import when, col

# # List of columns to replace Infinity values with NaN
# columns = ["Flow_Byts_s_imputed", "Flow_Pkts_per_s"]

# # Loop through each column and replace Infinity with NaN
# for col_name in columns:
#     df = df.withColumn(col_name, when(col(col_name) == float('inf'), float('nan')).otherwise(col(col_name)))


# output_cols = ["Flow_Byts_s_nan_imputed", "Flow_Pkts_per_s_nan_imputed"]


# #! impute nan values with mean
# imputer = Imputer(
#     inputCols=columns,
#     outputCols=output_cols
# ).setStrategy("median")

# df = imputer.fit(df).transform(df)

# df.describe("Flow_Byts_s_nan_imputed").show()

In [31]:
# VectorAssembler and the rest of the pipeline
from pyspark.ml.feature import VectorAssembler

feature_columns = [column for column in df.columns if column not in ['Label_binary']]  # Exclude the target column

vectorAssembler = VectorAssembler(inputCols=feature_columns, outputCol="features") # VectorAssembler

data = vectorAssembler.transform(df) # Transform the DataFrame

data = data.select('features', 'Label_binary')
data.show(5)


+--------------------+------------+
|            features|Label_binary|
+--------------------+------------+
|(78,[2,3,15,16,17...|           0|
|(78,[2,3,15,16,17...|           0|
|(78,[2,3,15,16,17...|           0|
|(78,[0,1,2,3,4,5,...|           0|
|(78,[0,1,2,3,4,5,...|           0|
+--------------------+------------+
only showing top 5 rows



In [32]:
# Check again for any null or NaN values that might have been introduced
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()


+--------+--------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+-------------+------------+------------+------------+-----------+------------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+--------------+--------------+-----------+-----------+------------+-----------+-----------+------------+------------+------------+------------+------------+------------+--------------+------------+-----------------+------------+----------------+----------------+------------------+------------------+----------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+-----------------+----------------+-------

In [33]:
# Sparse Vectors
data.show(10, truncate = False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|features                                                                                                                                                                                                                                                                                                                          

In [34]:
# Extract the values from Sparse Vectors to the list format
data_select = data.select('features').limit(2).collect()
for val in data_select:
    dense_vector = val[0].toArray()
    print(dense_vector)

[0.00000000e+00 0.00000000e+00 1.12641720e+08 3.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.63208600e+07
 1.39300034e+02 5.63209600e+07 5.63207600e+07 1.12641720e+08
 5.63208600e+07 1.39300034e+02 5.63209600e+07 5.63207600e+07
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 2.66331155e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.000000

### StandardScaler

In [35]:
# Normalizing feature vectors
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
scaler_model = scaler.fit(data)
data = scaler_model.transform(data)

data = data.select("scaledFeatures", "Label_binary")
data.show(3, truncate = False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|scaledFeatures                                                                                                                                                                                                                                                                                                                                                                                                                 |Label_binary|
+-----------------------------------------------------------------------------------------------------------------------------------------

In [36]:
# Split the data into training and testing sets
train_data, test_data = data.randomSplit([0.7, 0.3], seed = 42)

train_data.printSchema()

root
 |-- scaledFeatures: vector (nullable = true)
 |-- Label_binary: integer (nullable = false)



In [37]:
train_data.show(2, truncate = False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|scaledFeatures                                                                                                                                                

In [39]:
# ! Double check for any Negative, Infinity or NaN values

# Iterate through each column in the DataFrame
for column in df.columns:
    # Filter rows where the column contains problematic values
    problematic_rows = df.filter(
        (col(column).isNull()) |   # Check for null values
        (isnan(col(column))) |     # Check for NaN values
        (col(column) < 0)          # Check for negative values
        (col(column) == float('inf'))  # Check for infinity values
    )
    
    # If there are any rows containing problematic values, print the column name
    if problematic_rows.count() > 0:
        print(f"Column '{column}' has {problematic_rows.count()} rows with problematic values.")



---
# **Task 2 - Model Selection and Implementation (25 marks)**
---


### 1st student name: Philip Acquaye-Mensah
### Model: Logistic Regression

In [23]:
lr = LogisticRegression(featuresCol = "scaledFeatures", labelCol = 'Label_binary', threshold = 0.5, regParam = 0.01)

lr_model = lr.fit(train_data)

lr_predictions_train = lr_model.transform(train_data)
lr_predictions_test = lr_model.transform(test_data)




24/04/16 11:03:24 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [24]:
lr_predictions_test.select("Label_binary", "prediction").show(20) 

+------------+----------+
|Label_binary|prediction|
+------------+----------+
|           0|       0.0|
|           0|       0.0|
|           0|       0.0|
|           0|       0.0|
|           0|       0.0|
|           1|       1.0|
|           1|       1.0|
|           1|       1.0|
|           1|       1.0|
|           1|       1.0|
|           1|       1.0|
|           1|       1.0|
|           1|       1.0|
|           1|       1.0|
|           1|       1.0|
|           1|       1.0|
|           1|       1.0|
|           1|       1.0|
|           1|       1.0|
|           1|       1.0|
+------------+----------+
only showing top 20 rows



In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate AUC (Area Under ROC)
auc_evaluator = BinaryClassificationEvaluator(
    labelCol="Label_binary",
    rawPredictionCol="rawPrediction",  # ensure logistic regression is configured to provide rawPrediction
    metricName="areaUnderROC"
)
auc = auc_evaluator.evaluate(lr_predictions_test)
print("Area Under ROC = %g" % auc)
print("Area Under ROC : " , round(auc, 5)*100)

# Evaluate AUPRC (Area Under Precision-Recall Curve)
pr_evaluator = BinaryClassificationEvaluator(
    labelCol="Label_binary",
    rawPredictionCol="rawPrediction",  # ensure logistic regression is configured to provide rawPrediction
    metricName="areaUnderPR"
)
pr = pr_evaluator.evaluate(lr_predictions_test)
print("Area Under PR = %g" % auc)
print("Area Under PR = ",  round(pr, 5)*100)



Area Under ROC = 0.999922
Area Under ROC :  99.992


Area Under PR = 0.999922
Area Under PR =  99.971


2nd student name: Mohamed Jareer Mohamed Zeenam

## Apply Naive Bayes model

In [27]:
from pyspark.ml.classification import NaiveBayes

# Initialize Naive Bayes model
nb = NaiveBayes(featuresCol='scaledFeatures', labelCol='Label_binary', smoothing=1.0)  

# Fit the model to the training data
nb_model = nb.fit(train_data)

# Make predictions on the test data
nb_predictions = nb_model.transform(test_data)

In [28]:
nb_predictions.select('Label_binary', 'prediction').distinct().show(25)


24/04/15 13:34:29 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


+------------+----------+
|Label_binary|prediction|
+------------+----------+
|           0|       0.0|
|           1|       1.0|
|           0|       1.0|
|           1|       0.0|
+------------+----------+



#### Confusion Matrix

In [29]:
cm = nb_predictions.groupBy('Label_binary', 'prediction').count()
cm.show(cm.count(), truncate=False)

+------------+----------+------+
|Label_binary|prediction|count |
+------------+----------+------+
|0           |0.0       |177817|
|1           |1.0       |114293|
|0           |1.0       |22716 |
|1           |0.0       |3     |
+------------+----------+------+



In [30]:
#  Check the Missclassification rate
confusion_matrix = cm.groupBy('Label_binary').pivot('prediction').sum('count').na.fill(0)
confusion_matrix.show()

+------------+------+------+
|Label_binary|   0.0|   1.0|
+------------+------+------+
|           1|     3|114293|
|           0|177817| 22716|
+------------+------+------+



---
# **Task 3 - Model Parameter Tuning (20 marks)**
---


In [31]:
##1st student name: Philip Acquaye-Mensah
# add the code here


#### 2nd student name: Mohamed Jareer Mohamed Zeenam

##### Random Search results

In [32]:
import random
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Define the evaluator for binary classification
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Label_binary', metricName='areaUnderROC')

# Perform random search manually
best_auc_roc = 0.0
best_params = None

for smoothing in random.sample([0.1, 0.15, 0.2, 0.27, 0.3, 0.33, 0.4, 0.48, 0.5, 0.6, 0.7, 0.72, 0.76, 0.8, 0.9, 1], 5):
    nb = NaiveBayes(featuresCol='scaledFeatures', labelCol='Label_binary', smoothing=smoothing)
    model = nb.fit(train_data)

    # Make predictions
    predictions = model.transform(test_data)

    # Evaluate
    accuracy = evaluator.evaluate(predictions)

    # Check if the current model is the best so far
    if accuracy > best_auc_roc:
        best_auc_roc = accuracy
        best_params = smoothing




ERROR:root:KeyboardInterrupt while sending command.                 (0 + 4) / 4]
Traceback (most recent call last):
  File "/Users/philipacquaye-mensah/anaconda3/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/philipacquaye-mensah/anaconda3/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/philipacquaye-mensah/anaconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

---
# **Task 4 - Model Evaluation and Accuracy Calculation (20 marks)**
---

In [ ]:
##1st student name: Philip Acquaye-Mensah
# add the code here


---
# **Task 5 - Results Visualization or Printing (5 marks)**
---

In [ ]:
##1st student name: Philip Acquaye-Mensah
# add the code here


In [ ]:
##2nd student name: Mohamed Jareer MOHAMED  ZEENAM
# add the code here

# Print the results
print("Best smoothing value: ", best_params)
print("Area under ROC curve:", round(best_auc_roc, 5) * 100)
print("Accuracy:", round(accuracy, 5) * 100)

Best smoothing value:  0.33
Area under ROC curve: 94.33
Accuracy: 94.328


---
# **Task 6 - LSEP Considerations (5 marks)**
---

# Student 1: **Type the chosen issue**

add contribution here ...

# Student 2: **Type the chosen issue**

add contribution here ...

---

# **Task 7 - Convert ipynb to HTML for Turnitin submission [5 marks]**

---



In [ ]:
# install nbconvert (if facing the conversion error)
#!pip3 install nbconvert

In [ ]:
# convert ipynb to html and submit this HTML file
#!jupyter nbconvert --to html Your_Group_ID_CRWK_CN7030.ipynb

[NbConvertApp] WARNING | pattern 'Your_Group_ID_CRWK_CN7030.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]